In [1]:
import os
import uuid
import boto3
from typing import List, Literal
from pydantic import BaseModel, Field


from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.prompts import MessagesPlaceholder
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.tools.render import format_tool_to_openai_function 
from langchain.agents import tool
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.tools import BaseTool, Tool
from boto3.dynamodb.conditions import Key

OPENAI_API_KEY=os.environ['OPENAI_API_KEY']

In [31]:
class ExibirEBI(BaseModel):
    title: str = Field(description="Título do EBI resumido em poucas palavras")
    plaintext_base: str = Field(description="Texto base corrido, com versículos e referências")
    description: str = Field(description="10 Perguntas de observação, interpretação e aplicação, listadas em ordem numérica")
    footer: str = Field(description="Versículos de reflexão final")

In [38]:
@tool(args_schema=ExibirEBI)
def exibir_ebi(**dict_info:ExibirEBI) -> dict:
   """Chame essa função para exibir o EBI ao usuário"""
   print(dict_info["title"])
   print(dict_info["description"])
   print(dict_info["footer"])
   return "exibido com sucesso"

In [41]:
def generate_tools_for_user(session_id: str) -> List[BaseTool]:
    """Generate a set of tools that have a user id associated with them."""

    @tool(args_schema=ExibirEBI)
    def exibir_ebi(**dict_info:ExibirEBI) -> dict:
        """Chame essa função para exibir o EBI ao usuário"""
        dynamodb = boto3.resource('dynamodb')
        table = dynamodb.Table('EBITable')
        try:
            response = table.put_item(
                Item={'SessionId':f'{session_id}',"title":dict_info["title"],"base":dict_info["base"],"description":dict_info["description"],"footer":dict_info["footer"]})
            return "exibido com sucesso"
        except:
            return "aconteceu um erro ao exibir o EBI"
    
    return exibir_ebi

In [18]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('EBITable')
response = table.put_item(Item={'SessionId':'2','title':"dgfhg"})

In [21]:
id ='2'
exibir_ebi = generate_tools_for_user(id)

In [6]:
exibir_ebi.name

'exibir_ebi'

In [7]:
exibir_ebi.args_schema.schema_json()

'{"title": "ExibirEBI", "type": "object", "properties": {"title": {"title": "Title", "description": "T\\u00edtulo do EBI resumido em poucas palavras", "type": "string"}, "base": {"title": "Base", "description": "Texto base, com vers\\u00edculos e refer\\u00eancias", "type": "string"}, "description": {"title": "Description", "description": "10 Perguntas de observa\\u00e7\\u00e3o, interpreta\\u00e7\\u00e3o e aplica\\u00e7\\u00e3o, listadas em ordem num\\u00e9rica", "type": "string"}, "footer": {"title": "Footer", "description": "Vers\\u00edculos de reflex\\u00e3o final", "type": "string"}}, "required": ["title", "base", "description", "footer"]}'

In [22]:
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY,temperature=0.5, model="gpt-4o")

In [9]:
convert_to_openai_function(exibir_ebi)

{'name': 'exibir_ebi',
 'description': 'Chame essa função para exibir o EBI ao usuário',
 'parameters': {'type': 'object',
  'properties': {'title': {'description': 'Título do EBI resumido em poucas palavras',
    'type': 'string'},
   'base': {'description': 'Texto base, com versículos e referências',
    'type': 'string'},
   'description': {'description': '10 Perguntas de observação, interpretação e aplicação, listadas em ordem numérica',
    'type': 'string'},
   'footer': {'description': 'Versículos de reflexão final',
    'type': 'string'}},
  'required': ['title', 'base', 'description', 'footer']}}

In [33]:
model_with_tool = model.bind(functions=[convert_to_openai_function(exibir_ebi)])

In [34]:
conversation_table_name = "EBIsessionTable"
message_history = DynamoDBChatMessageHistory(table_name=conversation_table_name, session_id=id)
memory= ConversationBufferMemory(return_messages=True,memory_key="chat_history",chat_memory=message_history)

In [37]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """ Você é um especialista em teologia e em criação de estudo bíblico indutivo.
     Sua base bíblica é batista
     O estudo bíblico deve ser bem completo, com texto de referência, e com no mínimo 10 perguntas (Observação, interpretação e aplicação)
     Você cria estudo bíblicos voltados para jovens
     O estudo blíblico deve ter o texto corrido da passagem das escrituras, na versão nvi
     Você deve auxilixar o usuário a construir um EBI com base num texto bíblico
     Você deve sempre exibir o estudo bíblico, por meio de "exibir_ebi"
     Você não pode repetir ao usuário o EBI, apenas exiba por meio da ferramenta "exibir_ebi"
     você deve colocar o trecho do texto base no EBI 
     Caso o usuário solicite mudandas no EBI você deve executá-la e sempre exibir o estudo bíblico com a modificação
     """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [38]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model_with_tool | OpenAIFunctionsAgentOutputParser()

In [39]:
agent_executor= AgentExecutor(agent=agent_chain, tools=[exibir_ebi], verbose=True, memory=memory)

In [40]:
response=agent_executor.invoke({'input':"faça um estudo bíblico sobre o texto de lucas 12"})



> Entering new AgentExecutor chain...

Invoking: `exibir_ebi` with `{'title': 'Não se Preocupe com a Vida', 'base': "Lucas 12:22-34 (NVI)\n22 Então Jesus disse aos seus discípulos: 'Portanto eu lhes digo: não se preocupem com a sua própria vida, quanto ao que comer; nem com o seu próprio corpo, quanto ao que vestir. 23 A vida é mais importante do que a comida, e o corpo, mais do que as roupas. 24 Observem os corvos: não semeiam nem colhem, não têm armazéns nem celeiros; contudo, Deus os alimenta. E vocês têm muito mais valor do que as aves! 25 Quem de vocês, por mais que se preocupe, pode acrescentar uma hora que seja à sua vida? 26 Visto que vocês não podem sequer fazer uma coisa tão pequena, por que se preocupar com o restante? 27 Observem como crescem os lírios. Eles não trabalham nem tecem. Contudo, eu lhes digo que nem Salomão, em todo o seu esplendor, vestiu-se como um deles. 28 Se Deus veste assim a erva do campo, que hoje existe e amanhã é lançada ao fogo, quanto mais vestirá

In [42]:
response=agent_executor.invoke({'input':"refaça a pergunta 5"})



> Entering new AgentExecutor chain...

Invoking: `exibir_ebi` with `{'title': 'Não se Preocupe com a Vida', 'base': 'Lucas 12:22-34 (NVI)\n"22 Então Jesus disse aos seus discípulos: \'Portanto eu lhes digo: não se preocupem com a sua própria vida, quanto ao que comer; nem com o seu próprio corpo, quanto ao que vestir. 23 A vida é mais importante do que a comida, e o corpo, mais do que as roupas. 24 Observem os corvos: não semeiam nem colhem, não têm armazéns nem celeiros; contudo, Deus os alimenta. E vocês têm muito mais valor do que as aves! 25 Quem de vocês, por mais que se preocupe, pode acrescentar uma hora que seja à sua vida? 26 Visto que vocês não podem sequer fazer uma coisa tão pequena, por que se preocupar com o restante? 27 Observem como crescem os lírios. Eles não trabalham nem tecem. Contudo, eu lhes digo que nem Salomão, em todo o seu esplendor, vestiu-se como um deles. 28 Se Deus veste assim a erva do campo, que hoje existe e amanhã é lançada ao fogo, quanto mais vesti

In [35]:
agent_executor.memory.chat_memory.messages

[HumanMessage(content='faça um estudo bíblico sobre o texto de lucas 12'),
 AIMessage(content='O Estudo Bíblico Indutivo sobre Lucas 12 foi criado com sucesso. Aqui estão os detalhes:\n\n### Estudo Bíblico Indutivo sobre Lucas 12\n\n#### Texto Base\nLucas 12:1-34 (NVI)\n\n"Enquanto isso, ajuntando-se uma multidão de milhares de pessoas, a ponto de se atropelarem umas às outras, Jesus começou a falar, primeiramente aos seus discípulos, dizendo: \'Tenham cuidado com o fermento dos fariseus, que é a hipocrisia. Não há nada escondido que não venha a ser revelado, nem oculto que não venha a se tornar conhecido. O que vocês disseram na escuridão será ouvido à luz do dia, e o que sussurraram aos ouvidos dentro de casa será proclamado dos telhados.\'\n\nDigo a vocês, meus amigos: Não tenham medo dos que matam o corpo e depois nada mais podem fazer. Eu lhes mostrarei a quem vocês devem temer: Temam aquele que, depois de matar o corpo, tem poder para lançar no inferno. Sim, eu lhes digo, esse vo